In [7]:
from datasets import Dataset
import pandas as pd

dataset_path = "/Users/benjamyu/workspace/NLP-Letters/data/sentence_sets_trimmed.csv"
df = pd.read_csv(dataset_path, encoding='unicode_escape')
dataset = Dataset.from_pandas(df).train_test_split(test_size=0.2)

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['applicant_id', 'document', 'writer_gender', 'letter_type', 'applicant_gender', 'applicant_identify', 'applicant_identify_group', 'standardized_lor', 'us_canadian', 'usmle_1', 'usmle_2', 'applicants_writers', 'TEXT', 'full_text', 's1', 's2', 's1_s2', 's3', 'full_text_tokens'],
        num_rows: 2628
    })
    test: Dataset({
        features: ['applicant_id', 'document', 'writer_gender', 'letter_type', 'applicant_gender', 'applicant_identify', 'applicant_identify_group', 'standardized_lor', 'us_canadian', 'usmle_1', 'usmle_2', 'applicants_writers', 'TEXT', 'full_text', 's1', 's2', 's1_s2', 's3', 'full_text_tokens'],
        num_rows: 657
    })
})

In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

model_checkpoint = "distilbert-base-uncased"
batch_size = 16
task = "nlp-letters"
metric_name = "accuracy"
data_column = "full_text"
model_name = model_checkpoint.split("/")[-1]
num_labels = 2

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
)

def preprocess_function(sample):
    return tokenizer(sample[data_column], truncation=True)


encoded_dataset = dataset.map(preprocess_function, batched=True)
    
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    #compute_metrics=compute_metrics
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/2628 [00:00<?, ? examples/s]

Map:   0%|          | 0/657 [00:00<?, ? examples/s]

/Users/benjamyu/miniconda3/envs/nlp-letters/lib/python3.9/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [11]:
trainer.train()

ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,attention_mask.